# Prepare OpenCorporates Data with No States

The following code imports PatentsView data and the OpenCorporates API results to merge the two databases and identify organizations that can be found in both networks. OpenCorporates results in this script differ from previous versions because PatentsView states were not provided in the input file and any match between the two databases were matched only by organization name. Organization names in the OpenCorporates API results were cleaned to remove or standardize suffixes to facilitate better merge results between PatentsView and the OpenCorporates API results. This merge was performed to attach the unique assignee-state ID, location ID, city, and state from PatentsView to the OpenCorporates API results, and the data will be utilized to score the results. Scored records were normalized between a confidence level of 1-10.

In [1]:
### import the libraries used to process the PatentsView and OC data.
import pandas as pd
import numpy as np
import time
import os
import re
import string
import warnings
warnings.filterwarnings('ignore')

### start timer
t0=time.time()

### set the path for the input file and save to variable
res_folder = "../csvResults/"
input_file = "noState_output_cleaned.csv"
a_full=os.path.join(res_folder,input_file)
print(a_full,"\n")

OC_results=pd.read_csv(a_full)

### end timer and print total time
t1 = time.time()
total = t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset 
display(OC_results.info(null_counts=True),OC_results.head())

../csvResults/noState_output_cleaned.csv 

Total time is 0.001517 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 55 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    2623 non-null   int64  
 1   assignee_id                           2623 non-null   object 
 2   name                                  2034 non-null   object 
 3   company_number                        2034 non-null   object 
 4   jurisdiction_code                     2034 non-null   object 
 5   incorporation_date                    2034 non-null   object 
 6   dissolution_date                      551 non-null    object 
 7   company_type                          2018 non-null   object 
 8   registry_url                          1680 non-null   object 
 9   branch                                897 non-null    object 
 10  branch_status

None

,ID,assignee_id,name,company_number,jurisdiction_code,incorporation_date,dissolution_date,company_type,registry_url,branch,...,agent_street_address,agent_city,agent_state,agent_zipcode,home_company_name,home_company_jurisdiction_code,controlling_entity_name,controlling_entity_jurisdiction_code,list_of_officers,list_of_filing_dates
0,1,5d5ead2f-1ef7-4db6-a6ce-9cdea523f834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
1,2,8a841c57-22b6-4ad0-ad42-16532c3ab4fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
2,3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",68510F,us_ak,1/3/2000,NaN,Limited Liability Company,NaN,F,...,NaN,NaN,NaN,NaN,"LENDINGTREE, LLC",us_de,"LENDINGTREE, LLC",us_de,"['Christopher D. Davies', 'Douglas R Lebda', '...","['2011-07-21', '2011-07-21', '2008-03-17', '20..."
3,3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",1364151-0161,us_ut,6/27/1997,NaN,LLC - Foreign,https://secure.utah.gov/bes/details.html?entit...,F,...,NaN,NaN,NaN,NaN,"LENDINGTREE, LLC",us_de,"LENDINGTREE, LLC",us_de,"['INCORP SERVICES, INC']",[]
4,3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",34570412Q,us_la,8/29/1997,NaN,Limited Liability Company (Non-Louisiana),http://coraweb.sos.la.gov/commercialsearch/Com...,F,...,NaN,NaN,NaN,NaN,"LENDINGTREE, LLC",us_de,"LENDINGTREE, LLC",us_de,"['DOUGLAS LEBDA', 'INCORP SERVICES, INC']","['2017-04-06', '2015-10-18', '2013-01-28', '20..."


In [2]:
### construct a list of assignee_ids that did not have a matching record to the OC database. This
### will be used to manually search the OpenCorporates database and understand why the organizations
### are not matching
assigneesWithNoOCRecord = OC_results[OC_results['name'].isnull()].iloc[:,1:2].reset_index(drop=True)

### print general stats and first 5 records for dataset
display(assigneesWithNoOCRecord.info(),assigneesWithNoOCRecord.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   assignee_id  589 non-null    object
dtypes: object(1)
memory usage: 4.7+ KB


None

,assignee_id
0,5d5ead2f-1ef7-4db6-a6ce-9cdea523f834
1,8a841c57-22b6-4ad0-ad42-16532c3ab4fc
2,57ee3f8d-0756-4a84-a29d-adfdaea6e290
3,e809c8fa-1ae3-43b4-b504-307c633646e7
4,cb6a6b0b-4197-49a2-854d-d2a5e68e5847


In [3]:
### start timer
t0=time.time()

### set the path for the full PatentsView dataset and save to variable. This data will be used
### to assign the ID, location_id, cities, and states to the assigneesWithNoOCRecord variable
res_folder = "../csvResults/"
input_file = "dfMergedFullDataSet.csv"
a_full = os.path.join(res_folder,input_file)
print(a_full,"\n")

fullData = pd.read_csv(a_full)

### end timer and print total time
t1 = time.time()
total = t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(fullData.info(null_counts=True),fullData.head())

../csvResults/dfMergedFullDataSet.csv 

Total time is 0.007698 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274493 entries, 0 to 274492
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ID              274493 non-null  int64 
 1   assignee_id     274493 non-null  object
 2   location_id     274493 non-null  object
 3   organization    274493 non-null  object
 4   city            267595 non-null  object
 5   state           269716 non-null  object
 6   dateOfFirstPat  274493 non-null  object
dtypes: int64(1), object(6)
memory usage: 14.7+ MB


None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat
0,0,fffe9f1f-cb1c-49ab-b00f-6416e3e3a909,fd8b2b76-cb90-11eb-9615-121df0c29c1e,"Close-In Solutions, LLC",Austin,TX,10/18/2011
1,1,fffe8fea-3d13-4016-9429-93653527efa1,fe449928-09bd-11ec-893a-12de62d610b1,"Vita-Stat Neducak Services, Inc.",St. Petersburg,FL,6/10/1980
2,2,fffe4688-bc00-4626-bd89-28921a62f07f,f76d85c4-09bd-11ec-893a-12de62d610b1,"Emerging Technology Systems, L.L.C.",Akron,OH,1/6/1998
3,3,fffe36bb-6dea-4a8b-8bf5-071cf893ceba,fe1cb1c3-cb8f-11eb-9615-121df0c29c1e,"Valley Business Solutions, LLC",Huntsville,AL,6/30/2020
4,4,fffd9c21-3bb1-4471-b316-d172921e3f83,ec16f9be-cb90-11eb-9615-121df0c29c1e,"RAILIAS HOLDINGS, LLC",San Diego,CA,12/15/2020


# Create a List of Assignee IDs that Did Not Match to OpenCorporates when No States were Added to the Input File

In [4]:
### construct a list of assignees that had no matches to the OpenCorporates database by merging
### fullData with assigneesWithNoOCRecord. Duplicate assignee_ids were dropped to create a 1-1
### merge that used an inner join containing the records found in both sets based on assignee_id
fullDataDropDups=fullData.drop_duplicates(subset=['assignee_id'],keep='first')

assigneesWithNoOCRecord1=assigneesWithNoOCRecord.merge(fullDataDropDups,on=['assignee_id'],
                                                       how='inner').iloc[:,[1,0,2,3,4,5,6]].sort_values(by=['ID'])

### print general stats and first 5 records for dataset
display(assigneesWithNoOCRecord1.info(),assigneesWithNoOCRecord1.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 589 entries, 232 to 369
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              589 non-null    int64 
 1   assignee_id     589 non-null    object
 2   location_id     589 non-null    object
 3   organization    589 non-null    object
 4   city            566 non-null    object
 5   state           567 non-null    object
 6   dateOfFirstPat  589 non-null    object
dtypes: int64(1), object(6)
memory usage: 36.8+ KB


None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat
232,802,f8748ef9-f51b-4153-87ad-0f975f94caea,f9142a82-09be-11ec-893a-12de62d610b1,Wothington Cylinders Corporation,Worthington,OH,10/14/2014
469,942,f73af659-8c20-42e9-ac88-1dfdb155f2b9,fbb61109-cb8e-11eb-9615-121df0c29c1e,Steelcraft Fluorescent Company,Newark,NJ,2/12/1991
218,1670,f036b99a-e35f-4532-83c9-449a68d26b4c,ebd21fb5-cb8e-11eb-9615-121df0c29c1e,Burroughs Welcome Co.,Research Triangle Park,NC,2/20/1990
442,1824,eea554c2-8c0a-4d23-8601-320d7c841f98,d7a6bf2e-cb8f-11eb-9615-121df0c29c1e,"Variah Semiconductor Equipment Associates, Inc.",Gloucester,MA,7/20/2004
539,2098,ec0b2776-3780-4263-8e61-351e0393b656,d692d702-cb8f-11eb-9615-121df0c29c1e,Continuous Compostites Inc.,Coeur d'Alene,ID,8/18/2020


# Calculate the Percent of Assignee IDs Returned

In [5]:
### import the input file containing the 1000 records sent to Mike and calculate the % of assignees 
### with incorporation dates

### start timer
t0=time.time()

### set the path for the file that contains the 1000 records sent to the OpenCorporates API
### that tested how well the API works without states included in the input
res_folder = "../csvResults/"
input_file = "subsetToRerun_1000Records_noStates_Aug_3_2022.csv"
a_full = os.path.join(res_folder,input_file)
print(a_full,"\n")

pv1000NoStates=pd.read_csv(a_full)

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(pv1000NoStates.info(null_counts=True),pv1000NoStates.head())

../csvResults/subsetToRerun_1000Records_noStates_Aug_3_2022.csv 

Total time is 0.000083 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   assignee_id     1000 non-null   object
 1   organization    1000 non-null   object
 2   dateOfFirstPat  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


None

,assignee_id,organization,dateOfFirstPat
0,5d5ead2f-1ef7-4db6-a6ce-9cdea523f834,Vitric Corporation,7/27/1999
1,8a841c57-22b6-4ad0-ad42-16532c3ab4fc,American Bicycle Manufacturing Company,9/29/1987
2,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LendingTree, LLC",5/7/2002
3,b4f82aab-7c15-4868-8044-81a0ed5d33be,"Nordstrom Valves, Inc",1/11/2000
4,d62b9811-b171-40ed-a739-7eb54199f188,"LT 350, LLC",5/9/2017


In [6]:
### save the OC_results to another variable to ensure the original variable is not overwritten
OC_results2=OC_results

### drop the records that do not contain an incorporation date. This set will be used to calcualte
### the % of return assignee_ids.
OC_results2.dropna(subset=['incorporation_date'],inplace=True)
OC_results3=OC_results2.iloc[:,[1,2,4,5,20,21,33,42,43,46,47]]

### print general stats and first 5 records for dataset
display(OC_results3.info(),OC_results3.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 2 to 2622
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   assignee_id         2034 non-null   object
 1   name                2034 non-null   object
 2   jurisdiction_code   2034 non-null   object
 3   incorporation_date  2034 non-null   object
 4   alternative_names   288 non-null    object
 5   previous_names      496 non-null    object
 6   data                985 non-null    object
 7   address_city        976 non-null    object
 8   address_state       961 non-null    object
 9   agent_city          292 non-null    object
 10  agent_state         290 non-null    object
dtypes: object(11)
memory usage: 190.7+ KB


None

,assignee_id,name,jurisdiction_code,incorporation_date,alternative_names,previous_names,data,address_city,address_state,agent_city,agent_state
2,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",us_ak,1/3/2000,NaN,"[{'company_name': 'LENDINGTREE, INC.'}]",NaN,CHARLOTTE,NC,NaN,NaN
3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",us_ut,6/27/1997,"[{'company_name': 'SERVICEMAGIC HOME LOANS', '...","[{'company_name': 'CREDITSOURCE USA, INCORPORA...",NaN,CHARLOTTE,NC,NaN,NaN
4,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",us_la,8/29/1997,NaN,"[{'company_name': 'LENDINGTREE, INC.', 'end_da...","{'most_recent': [{'datum': {'id': 37370767, 't...",BATON ROUGE,LA,NaN,NaN
5,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",us_me,6/7/1996,[{'company_name': 'REALESTATE.COM - CANCELLED'...,NaN,"{'most_recent': [{'datum': {'id': 114822113, '...",NaN,NaN,NaN,NaN
6,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,"LENDINGTREE, LLC",us_ut,6/27/1997,"[{'company_name': 'GETSMART', 'type': 'trading...","[{'company_name': 'LENDINGTREE, INC.'}, {'comp...",NaN,CHARLOTTE,NC,NaN,NaN


In [7]:
### calculate the % of unique returned assignee_ids. This is done by taking the total number of unique
### assignee_ids with an incorporation date in the OpenCorporates output file and dividing by the total
### number of unique assignee_ids from the input file. In the below calculations, the total number of 
### unique output and input assignee_ids are printed followed by the percent of unique assignee_ids
### calculation
print("The number of unique assignee IDs in the output file is", OC_results3['assignee_id'].nunique())
print("The number of unique assignee IDs returned in the input file is", pv1000NoStates['assignee_id'].nunique())
print(f"The percent of returned assignee_ids is {(OC_results3['assignee_id'].nunique())/pv1000NoStates['assignee_id'].nunique():.2%}")

The number of unique assignee IDs in the output file is 411
The number of unique assignee IDs returned in the input file is 1000
The percent of returned assignee_ids is 41.10%


# Standardize the Organization Names from the PatentsView Data and OpenCorporates Results

In [8]:
### this section cleans the organization and name fields by standardizing the names for the
### organizations between the PatentsView and OpenCorporates data to facilitate better merge
### results. Skipping this section will yield poor results in any subsequent merges.

### start timer
t0=time.time()

### the pattern variable is utilized to create a 'list' of possible suffixes that should be removed
### from the organization and name fields. This list was constructed by manually inspecting the names
### prior to cleaning and does not represent a comprehensive list
pattern = '|'.join(['Llc','L\.L\.C\.','Inc\.$','Inc$','Ltd','\(.+?\)','Plc','P\.L\.C\.','Pllc','P\.L\.L\.C\.',
                    'Lp\.$','Lp$','Llp$','LP','L\.P\.','LC','L\.C\.','Ag$','Gmbh','SA$','Kg','Pvt','Sa$','BV','Nv$',
                    'Ab$','Pty$','SPA$','S\.P\.A\.','Bv','B\.V\.','B\.v\.','@','\.',','])

### converts the first character in each word to Uppercase and remaining characters to Lowercase in 
### the string, followed by removing any whitespace that may exist to the left and right of the strings
fullData['organization']=fullData['organization'].str.title()
fullData['organization']=fullData['organization'].str.lstrip().str.rstrip()

### some strings must be replaced rather than removed because the resulting organization names would
### not make sense or match incorrectly. For example, Arjang & Co., which is the full name for the
### organization, would become Arjang and would match to multiple records via the merge instead of
### one. This was observed through multiple trials of cleaning the data
fullData['organization']=fullData['organization'].str.replace(' & ', ' And ')
fullData['organization']=fullData['organization'].str.replace('&', ' And ')
fullData['organization']=fullData['organization'].str.replace(' - |-', ' ')
fullData['organization']=fullData['organization'].str.replace('+', ' ')
fullData['organization']=fullData['organization'].str.replace(' (Co\.$|Co$)', ' Company')
fullData['organization']=fullData['organization'].str.replace(' (Corp\.|Corp) ', ' Corporation ')
fullData['organization']=fullData['organization'].str.replace('Mfg', 'Manufacturing')
fullData['organization']=fullData['organization'].str.replace('Incorporated|Usa', '')

### apply the pattern variable to the organization field and clean the resulting whitespace to the
### left and right of the strings
fullData['organization']=fullData['organization'].str.replace(pattern, '')
fullData['organization']=fullData['organization'].str.lstrip().str.rstrip()

### convert any remaining names that are not standardized
fullData['organization']=fullData['organization'].str.replace(' (Co\.$|Co$)', ' Company')
fullData['organization']=fullData['organization'].str.replace(' (Corp\.$|Corp$) ', ' Corporation ')
fullData['organization']=fullData['organization'].str.replace(' (Corp\.$|Corp$)', ' Corporation')

### converts the first character in each word to Uppercase and remaining characters to Lowercase in 
### the string, followed by removing any whitespace that may exist to the left and right of the strings
OC_results3['name']=OC_results3['name'].str.title()
OC_results3['name']=OC_results3['name'].str.lstrip().str.rstrip()

### some strings must be replaced rather than removed because the resulting organization names would
### not make sense or match incorrectly. For example, Arjang & Co., which is the full name for the
### organization, would become Arjang and would match to multiple records via the merge instead of
### one. This was observed through multiple trials of cleaning the data
OC_results3['name']=OC_results3['name'].str.replace(' & ', ' And ')
OC_results3['name']=OC_results3['name'].str.replace('&', ' And ')
OC_results3['name']=OC_results3['name'].str.replace(' - |-', ' ')
OC_results3['name']=OC_results3['name'].str.replace('+', ' ')
OC_results3['name']=OC_results3['name'].str.replace(' (Co\.$|Co$)', ' Company')
OC_results3['name']=OC_results3['name'].str.replace(' (Corp\.|Corp) ', ' Corporation ')
OC_results3['name']=OC_results3['name'].str.replace('Mfg', 'Manufacturing')
OC_results3['name']=OC_results3['name'].str.replace('Incorporated|Usa', '')

### apply the pattern variable to the name field and clean the resulting whitespace to the
### left and right of the strings
OC_results3['name']=OC_results3['name'].str.replace(pattern, '')
OC_results3['name']=OC_results3['name'].str.lstrip().str.rstrip()

### convert any remaining names that are not standardized
OC_results3['name']=OC_results3['name'].str.replace(' (Co\.$|Co$)', ' Company')
OC_results3['name']=OC_results3['name'].str.replace(' (Corp\.$|Corp$) ', ' Corporation ')
OC_results3['name']=OC_results3['name'].str.replace(' (Corp\.$|Corp$)', ' Corporation')

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

Total time is 0.043156 mins


# Extract Alternative and Previous Organization Names and Append to the OpenCorporates Result Dataset

In [9]:
### consecutive regex processes were performed to extract one or more organization names from the
### alternative_names and previous_names columns. When a match is absent, the if statement skips
### that record. If a match is found, the first regex variable is used to find all instances that
### match the pattern, then the second regex variable is utilized to further refine the string to
### reduce it to the organization name. A list is created using the nested for loop for each record.
### If multiple names are present, a nested list is created for that record. After the for loops 
### are finished running, the final list is created and printed for reviewing. This description
### applies for processing both the alternative_names and previous_names columns.

### start timer
t0=time.time()

### converts the first character in each word to Uppercase and remaining characters to Lowercase in 
### the string
OC_results3['alternative_names']=OC_results3['alternative_names'].str.title()
OC_results3['previous_names']=OC_results3['previous_names'].str.title()

### regex and regex1 were applied to initially find a list of organization names, followed by refining
### the names by removing erroneous characters
regex="'Company_Name': '[A-Z].+?'"
regex1="'[A-Z].+': "

### determines the length of the input data and creates 2 empty lists to be filled with the extracted
### organization names
b=len(OC_results3)
sub_finalAlt = []
finalAlt = []

### the for loop is initiated to review each record individually to extract the organization names
### under the alternative_names column
for j in range(b):
    
    ### if the record is empty or nan the record will be skipped
    if pd.isna(OC_results3.iloc[j,4]) is True:
        pass
    
    ### non-empty records have the curly brackets removed from the right and left of the list of
    ### strings. The findall function searches the list for all instances that match the regex
    ### variable and saves the list to the match variable. The length of this list is determined
    ### and provide to the nested loop below
    else:
        a=OC_results3.iloc[j,4][2:-2]
        match = re.findall(regex, a)
        c=len(match)
        
        ### for any length of c greater than 0, this for loop will evaluate each instance for the
        ### organization name by using the regex1 variable to remove characters that are not
        ### required. Once a match is found, the first character in each string is converted to
        ### an uppercase and saved to the sub_finalAlt list. The for loop repeats as many times
        ### that is equal to c
        for i in range(c):
            match1 = re.sub(regex1,"",match[i])
            match1=match1.replace("'","").title()
            
            ### ensures all spaces between strings are a single space
            match1 = re.sub('\s+',' ',match1)
            
            ### replaces specific characters
            match1=re.sub(' & ', ' And ',match1)
            match1=match1.replace('&', ' And ')
            match1=re.sub('Mfg', 'Manufacturing',match1)

            ### convert any names that are not standardized
            match1=re.sub(', Inc\.| Usa, Incorporated| Usa| Inc\.| Inc$| Incorpor$| Incorporated$|Incorporated', '',match1)
            match1=re.sub(', Llc$| Ltd\.| Ltd| Limited| Pty\.| Pty', '',match1)
            match1=re.sub('L\.L\.C\.| Co\.$| Llp$| Co$| Llc$|\(.+?\)|,Inc\.', '',match1)
            match1=re.sub(', P\.C\.| P\.C\.| P\. C\.| D\.M\.D\.|D\.D\.S\.| D\. D\. S\.| M\.D\.', '',match1)
            match1=re.sub(' Corporation$| Corporation,$| Corp\.$ | Corp.$| Corp\.$| Corp\.,$', 'Corporation',match1)
            match1=re.sub('Co\.$| Co$', 'Company',match1)
            
            ### removes all punctuation and ensures all spaces were single spaced after processing
            ### the strings
            match1 = match1.translate(str.maketrans('','',string.punctuation))
            match1 = re.sub('\s+',' ',match1)
            
            ### appends the strings to list
            sub_finalAlt.append(match1)
    
    ### after the nested for loop is finished extracting all possible organization names, the
    ### sub_finalAlt list is appended to the finalAlt list. The finalAlt list is updated with 
    ### the alternative names for each record after the for loops are finished and resets the
    ### sub_finalAlt to an empty list
    finalAlt.append(sub_finalAlt)
    sub_finalAlt = []

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins\n")

### prints the finalAlt list for review
print(finalAlt)

Total time is 0.002183 mins

[[], ['Servicemagic Home Loans', 'RealestateCom Realtors', 'Servicemagic Real Estate', 'Lendingtree PartnersCom', 'Magnifymoney', 'Getsmart', 'RealestateCom', 'Depositaccounts', 'Onlinebanks', 'Ramsey Group A Real EstateCom Company', 'Milecards', 'Student Loan Hero Of Ut', 'GetsmartCom', 'Snapcap'], [], ['RealestateCom Cancelled', 'GetsmartCom', 'Servicemagic Real Estate', 'Servicemagic Home Loans', 'Getsmart', 'RealestateCom Realtors Cancelled', 'Lendingtree PartnersCom', 'Depositaccounts', 'Magnifymoney', 'Milecards', 'Snapcap', 'Student Loan Hero'], ['Getsmart', 'Servicemagic Real Estate', 'GetsmartCom', 'Ramsey Group A Real EstateCom Company', 'Snapcap', 'RealestateCom', 'Servicemagic Home Loans', 'Onlinebanks', 'Lendingtree PartnersCom', 'Depositaccounts', 'Milecards', 'Student Loan Hero Of Ut', 'RealestateCom Realtors', 'Magnifymoney'], [], [], ['Nordstrom Audco', 'Nordstrom Valves'], [], [], [], [], [], ['Minop'], [], [], [], [], [], [], [], [], ['Le

In [10]:
### start timer
t0=time.time()

### creates 2 empty lists to be filled with the extracted organization names
sub_finalPre = []
finalPre = []

regex="'Company_Name': '[A-Za-z].+?'"
regex1="'[A-Za-z].+': "

### the for loop is initiated to review each record individually to extract the organization names
### under the previous_names column
for j in range(b):
    
    ### if the record is empty or nan the record will be skipped
    if pd.isna(OC_results3.iloc[j,5]) is True:
        pass
    
    ### non-empty records have the curly brackets removed from the right and left of the list of
    ### strings. The findall function searches the list for all instances that match the regex
    ### variable and saves the list to the match variable. The length of this list is determined
    ### and provide to the nested loop below
    else:
        a=OC_results3.iloc[j,5][2:-2]
        match = re.findall(regex, a)
        c=len(match)

        ### for any length of c greater than 0, this for loop will evaluate each instance for the
        ### organization name by using the regex1 variable to remove characters that are not
        ### required. Once a match is found, the first character in each string is converted to
        ### an uppercase and saved to the sub_finalPre list. The for loop repeats as many times
        ### that is equal to c
        for i in range(c):
            match1 = re.sub(regex1,"",match[i])
            match1=match1.replace("'","").title().lstrip().rstrip()
            
            ### ensures all spaces between strings are a single space
            match1 = re.sub('\s+',' ',match1)
            
            ### replaces specific characters
            match1=re.sub(' & ', ' And ',match1)
            match1=match1.replace('&', ' And ')
            match1=re.sub('Mfg', 'Manufacturing',match1)

            ### convert any names that are not standardized
            match1=re.sub(', Inc\.| Usa, Incorporated| Usa| Inc\.| Inc$| Incorpor$| Incorporated$|Incorporated', '',match1)
            match1=re.sub(', Llc$| Ltd\.| Ltd| Limited| Pty\.| Pty', '',match1)
            match1=re.sub('L\.L\.C\.| Co\.$| Llp$| Co$| Llc$|\(.+?\)|,Inc\.', '',match1)
            match1=re.sub(', P\.C\.| P\.C\.| P\. C\.| D\.M\.D\.|D\.D\.S\.| D\. D\. S\.| M\.D\.', '',match1)
            match1=re.sub(' Corporation$| Corporation,$| Corp\.$ | Corp.$| Corp\.$| Corp\.,$', 'Corporation',match1)
            match1=re.sub('Co\.$| Co$', 'Company',match1)
            
            ### removes all punctuation and ensures all spaces were single spaced after processing
            ### the strings
            match1 = match1.translate(str.maketrans('','',string.punctuation))
            match1 = re.sub('\s+',' ',match1)

            sub_finalPre.append(match1)

    ### after the nested for loop is finished extracting all possible organization names, the
    ### sub_finalPre list is appended to the finalAlt list. The finalPre list is updated with 
    ### the previous names for each record after the for loops are finished and resets the
    ### sub_finalPre to an empty list
    finalPre.append(sub_finalPre)
    sub_finalPre = []

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins\n")

### prints the finalPre list for review
print(finalPre)

Total time is 0.002133 mins

[['Lendingtree'], ['Creditsource', 'Lendingtree'], ['Lendingtree', 'Creditsource'], [], ['Lendingtree', 'Creditsource'], [], ['Creditsource', 'Lending Tree'], [], [], [], [], [], [], [], [], [], ['Carson And Burger', 'Carson Burger And Weekly', 'Cbw Automation'], [], [], [], [], ['Las'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['Las Enterprises'], [], [], [], [], [], [], [], [], ['Molecular Simulations', 'PolygenCorporation'], [], [], [], [], ['Powervar Canada', 'Twin City Computers'], [], ['Jibjab Media'], ['Jibjab Media'], [], [], [], [], ['Dielectrics Industries', 'Dielectrics'], [], [], [], [], [], [], ['Paul Revere TobaccoCorporation'], [], ['Paul Revere Equity Management'], ['Paul Revere Equity Management Company The'], ['Wake Up America The Reds Are Coming Paul RevereJoeCorporation'], [], [], [], [], ['Ici Ceramics'], [], [], [], [], [], ['Ramer Products', 'Ramer Test Tools', 'Test Tools', 'Test Tools'], []

In [11]:
### start timer
t0=time.time()

### append the OC_results dataset with the finalAlt and finalPre lists with a _clean suffix
OC_results3['alternative_names_clean'] = finalAlt
OC_results3['previous_names_clean'] = finalPre

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(OC_results3.info(),OC_results3.head())

Total time is 0.000050 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 2 to 2622
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   assignee_id              2034 non-null   object
 1   name                     2034 non-null   object
 2   jurisdiction_code        2034 non-null   object
 3   incorporation_date       2034 non-null   object
 4   alternative_names        288 non-null    object
 5   previous_names           496 non-null    object
 6   data                     985 non-null    object
 7   address_city             976 non-null    object
 8   address_state            961 non-null    object
 9   agent_city               292 non-null    object
 10  agent_state              290 non-null    object
 11  alternative_names_clean  2034 non-null   object
 12  previous_names_clean     2034 non-null   object
dtypes: object(13)
memory usage: 222.5+ KB


None

,assignee_id,name,jurisdiction_code,incorporation_date,alternative_names,previous_names,data,address_city,address_state,agent_city,agent_state,alternative_names_clean,previous_names_clean
2,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ak,1/3/2000,NaN,"[{'Company_Name': 'Lendingtree, Inc.'}]",NaN,CHARLOTTE,NC,NaN,NaN,[],[Lendingtree]
3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ut,6/27/1997,"[{'Company_Name': 'Servicemagic Home Loans', '...","[{'Company_Name': 'Creditsource Usa, Incorpora...",NaN,CHARLOTTE,NC,NaN,NaN,"[Servicemagic Home Loans, RealestateCom Realto...","[Creditsource, Lendingtree]"
4,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_la,8/29/1997,NaN,"[{'Company_Name': 'Lendingtree, Inc.', 'End_Da...","{'most_recent': [{'datum': {'id': 37370767, 't...",BATON ROUGE,LA,NaN,NaN,[],"[Lendingtree, Creditsource]"
5,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_me,6/7/1996,[{'Company_Name': 'Realestate.Com - Cancelled'...,NaN,"{'most_recent': [{'datum': {'id': 114822113, '...",NaN,NaN,NaN,NaN,"[RealestateCom Cancelled, GetsmartCom, Service...",[]
6,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ut,6/27/1997,"[{'Company_Name': 'Getsmart', 'Type': 'Trading...","[{'Company_Name': 'Lendingtree, Inc.'}, {'Comp...",NaN,CHARLOTTE,NC,NaN,NaN,"[Getsmart, Servicemagic Real Estate, GetsmartC...","[Lendingtree, Creditsource]"


In [12]:
### drop the alternative_names and previous_names columns and rearrange the alternative_names_clean 
### and previous_names_clean features
OC_results4=OC_results3
OC_results4.drop(labels=['alternative_names','previous_names'],axis=1,inplace=True)

ele_col = OC_results4.pop('alternative_names_clean')
twe_col = OC_results4.pop('previous_names_clean')

OC_results4.insert(4, 'alternative_names_clean', ele_col)
OC_results4.insert(5, 'previous_names_clean', twe_col)

### print general stats and first 5 records for dataset
display(OC_results4.info(),OC_results4.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 2 to 2622
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   assignee_id              2034 non-null   object
 1   name                     2034 non-null   object
 2   jurisdiction_code        2034 non-null   object
 3   incorporation_date       2034 non-null   object
 4   alternative_names_clean  2034 non-null   object
 5   previous_names_clean     2034 non-null   object
 6   data                     985 non-null    object
 7   address_city             976 non-null    object
 8   address_state            961 non-null    object
 9   agent_city               292 non-null    object
 10  agent_state              290 non-null    object
dtypes: object(11)
memory usage: 190.7+ KB


None

,assignee_id,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,data,address_city,address_state,agent_city,agent_state
2,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ak,1/3/2000,[],[Lendingtree],NaN,CHARLOTTE,NC,NaN,NaN
3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ut,6/27/1997,"[Servicemagic Home Loans, RealestateCom Realto...","[Creditsource, Lendingtree]",NaN,CHARLOTTE,NC,NaN,NaN
4,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_la,8/29/1997,[],"[Lendingtree, Creditsource]","{'most_recent': [{'datum': {'id': 37370767, 't...",BATON ROUGE,LA,NaN,NaN
5,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_me,6/7/1996,"[RealestateCom Cancelled, GetsmartCom, Service...",[],"{'most_recent': [{'datum': {'id': 114822113, '...",NaN,NaN,NaN,NaN
6,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ut,6/27/1997,"[Getsmart, Servicemagic Real Estate, GetsmartC...","[Lendingtree, Creditsource]",NaN,CHARLOTTE,NC,NaN,NaN


# Extract Additional City and States from the Data Field and Append to the OpenCorporates Dataset

In [14]:
### this library allows the user to remove all punctuation in a string
import string 

### start timer
t0=time.time()

### the regex variables are applied to extract substrings in the data field and sequentially processed
### to remove most of the erroneous data
regex  = "'description': '[A-Za-z0-9].+?'"
regex1 = ": '[A-za-z0-9].+': "

### the b variable sets the length of the input data and the list of states will be used to extract the
### states from the data being processed. The empty lists are populated with the extracted data and will
### be utilized appended to the input dataframe
b=len(OC_results4)
stateList = "Al$|Ak|Az$|Ar$|Ca$|Co$|Ct$|Dc$|De$|Fl$|Ga$|Hi$|Id$|Il$|In$|Ia$|Ky$|Ks$|La$|Me$|Ma$|Md$|Mi$|Mn$|Ms$|Mo$| \
             Mt$|Ne$|Nv$|Nh$|Nj$|Nm$|Ny$|Nc$|Nd$|Oh$|Ok$|Or$|Pa$|Ri$|Sc$|Sd$|Tn$|Tx$|Ut$|Vt$|Va$|Wa$|Wv$|Wi$|Wy$|Fl$"
sub_city = []
city = []
sub_states1 = []
states = []

### this for loop initiates the search for city-state pairs within the data field. If there was no data
### found in the cell (Nan or Na), the if statement enters a blank for that record and moves to the next
### record. When a record was identified to contain data, the regex variable was applied with the findall
### function to extract all data matching the pattern. Lastly, the length of the match variable was
### determined and passed to the nexted for loop
for j in range(b):
    
    ### enters a blank for the record if no data is present
    if pd.isna(OC_results4.iloc[j,6]) is True:
        sub_states1 = []
        sub_city= []

    else:
        ### select the non-empty record and apply the findall function to obtain all matches, then
        ### calculate the length of the resulting string(s)
        a=OC_results4.iloc[j,6]
        match=re.findall(regex,a)
        c=len(match)

        ### each instance of city-state pairs identified in the findall function above were processed and
        ### standardized (e.g., city name two-letter state abbreviation). The following does not represent
        ### a comprehensive list of standardizing city-states
        for i in range(c):
            match1 = re.sub(regex1,"",match[i])
            match1 = re.sub('[0-9]',"",match1)
            match1 = re.sub('[0-9]',"",match1)
            match1 = re.sub('\s+',' ',match1)
            match1 = match1.replace("'","").title()
            match1 = match1.replace('Description: ',"")
            match1 = match1.replace('\\N'," ")
            match1 = match1.replace('\\n'," ")
            match1 = match1.replace('-',"")
            match1 = match1.replace('#',"")
            match1 = re.sub('Us$|USA|Usa|United States Of America|United States|Register Id:',"",match1)
            match1 = re.sub('Suite|Ste',"",match1)
            match1 = re.sub('Http.+\.[Cc]om',"",match1)
            match1 = re.sub(',$',"",match1)
            match1 = re.sub('/|:',"",match1)
            match1 = re.sub('[A-z].+(Department|Dept)\.|[A-z].+Dept',"",match1)
            match1 = re.sub('[A-z].+Bates',"",match1)
            match1 = re.sub('[A-z].+Larocque,',"",match1)
            match1 = re.sub('Apt ([A-z]|[A-z]\.)|Apt\.',"",match1)
            match1 = re.sub('[Pp].+?Box,|[Pp].+?Box ,|Po ,|Box , ',"",match1)
            match1 = re.sub('[A-z].+?Cook,',"",match1)
            match1 = re.sub('[A-z].+?Compliance, ',"",match1)
            match1 = re.sub('Th Floor, |Th Fl|Th Fl,',"",match1)
            match1 = re.sub('[A-z].+?Nova, ',"",match1)
            match1 = re.sub('[A-z].+?Ave,',"",match1)
            match1 = re.sub('[A-z].+?(Center|Ctr),',"",match1)
            match1 = re.sub('[A-z].+(Lane|Ln)',"",match1)            
            match1 = re.sub('(Bldg|Bldg\.) ([A-z]|[A-z],|[A-z]\.)|Bldg\.|Bldg',"",match1)
            match1 = re.sub('(Rd|Rd\.) (Floor,|Fl\.,|Fl,)',"",match1)
            match1 = re.sub('[A-Z].+?(Road|Rd\.|Rd)',"",match1)
            match1 = re.sub('[A-Z].+?Blvd|%Comet Glass Co, |Lcr ',"",match1)
            match1 = re.sub('[A-Z].+?Capitol| , By Corinne M Lude|(B,|B) |C, |^r, |^Z |^z ',"",match1)
            match1 = re.sub('[A-Z].+?Accounting, |[A-Z].+?Siuta, |Santa Helena|Fl , |E Th Saint ',"",match1)          
            match1 = re.sub('N Bay Village Fla',"North Bay Village, Fl",match1)
            match1 = re.sub('Mt\.|Mt',"Mount",match1)
            match1 = re.sub('Ft\.|Ft',"Mount",match1)
            match1 = re.sub('St\. |St ',"Saint ",match1)
            match1 = re.sub('Spgs',"Springs",match1)
            match1 = re.sub('Th Avenue South East, Mpls, Mn',"Minneapolis, Mn",match1)
            match1 = re.sub('No. Adams,, Ma, ',"North Adams, Ma",match1)
            match1 = re.sub('N Palm Beach, Fl',"North Palm Beach, Fl",match1)
            match1 = re.sub('Lk',"Lake",match1)
            match1 = re.sub('.+?(Pkwy|Parkway)|.+?Saint E, ',"",match1)
            match1 = re.sub('^Inc.+?(Place|Pl) |.+?(Place|Pl), |.+?(Plaza|Plz) ',"",match1)
            match1 = re.sub('Inc\., Irving Pl,',"",match1)
            match1 = re.sub('^Inc.+?(Street|St)|Inc.  N Frederick Ave L',"",match1)
            match1 = re.sub('.+?(Street|Street,|St\.,|St,) ',"",match1)
            match1 = re.sub('^(Inc|Inc\.,).+?(Drive|(Dr,|Dr)) |.+?(Highway|Hwy) |.+?(Freeway|Fwy) |.+?(Way|Wy) ',"",match1)
            match1 = re.sub('.+?(Drive|(Dr,|Dr|Dr\.,|Dr\.)|Drive,) |N Mi Saint',"",match1)
            match1 = re.sub('Glenroy|N Central Ave|Ne, |Nw |.+?Counsel, |.+?(Boulevard,|Boulevard) |Loockerman Square ',"",match1)
            match1 = re.sub('Lafox',"La Fox",match1)
            match1 = re.sub('Plymo$',"Plymouth",match1)
            match1 = re.sub('Portl$',"Portland",match1)
            match1 = re.sub('Flore$',"Florence",match1)
            match1 = re.sub('Southfi$',"Southfield",match1)
            match1 = re.sub('.+?(Longwood|Longwood,) Fl',"Longwood Fl",match1)
            match1 = re.sub('.+?p\.O\. Box|.+?(Highway|Hwy), |Inc. > box|P O Box |%.+?, |.+?(Avenue|Ave) ',"",match1)
            match1 = re.sub('^oor |Inc\.,|.+?(Court|Ct\.), |.+?(Avenue|Ave|Ave\.,), |.+?Ave |Inc\.  El Camino Real',"",match1)
            match1 = re.sub('E&A ennedy Space Ctr.,|E&A , Kennedy Space Ctr.,',"Kennedy Space Center,",match1)
            match1 = re.sub('.+?South Bend, In',"South Bend, In",match1)
            match1 = re.sub('.+?indio Ca',"Indio Ca",match1)
            match1 = re.sub('.+?[Ss]ioux Falls, ',"Sioux Falls, ",match1)
            match1 = re.sub('.+?(Plaza|Plz), |North Th Saint |.+?Belmont, ',"",match1)
            match1 = re.sub('.+?Belleville, Il,',"Belleville Il",match1)
            match1 = re.sub('Woodlawn Saint|s,|Spiceland|.+?(Circle|Cir)|Hillsboro Mile|Saint ,|.+?(Way|Wy)|N Court Saint',"",match1)
            match1 = re.sub('W\. Adam|Heritage Hill|Franklin Saint|.+?[Tt]ower |Pecan Saint W|Curti |.+?(Park|Pk) |S\.W Th',"",match1)
            match1 = re.sub('E Th Saint |Harlin Sr|Pmb|N\. Raymond|S Th|.+?(Pike|Pk)|.+?(Court|Ct)|.+?(Department|Dept)',"",match1)
            match1 = re.sub('West Tenth Saint|.+?Broadway|Boradway|Rr Box R|.+?(Route|Rte)|Box S\.|cl,|El Rio Saint',"",match1)
            match1 = re.sub('Village Square|Orange Saint|N Causeway|Plone Et Al|Madison Saint|P O Drawer Www',"",match1)
            match1 = re.sub('Po Bo|Chisholm Pl|.+?Ave\., |Prescott Saint|Titian|Picacho|Rusk Saint|Market Saint',"",match1)
            match1 = re.sub('Dairy Ashford Saint|Reichhold|Wall Saint|Webster Saint Fl Th|Webster Saint',"",match1)
            match1 = re.sub('(West|W) El Camino Real|Mountain (Trail|Trl)|W Th Saint  E|Rocky Mountain Fiber',"",match1)
            match1 = re.sub('Lake Saint Loui Mo',"Saint Louis Mo",match1)
            match1 = re.sub('Redhill Ave|Th Saint  |Th Saint ste|S\. Akard Saint|Sw Third |Rm |.+?(Loop|Loop,) |Alcor Body Work',"",match1)
            match1 = re.sub('.+?(Boulevard|Bvld)|Ave\., |.+?Solution|Braodway|Larkspur|.+?Turnpike|ele ,|Acorn Saint',"",match1)
            match1 = re.sub('.+?(Martius|Martiu) |Brush Saint |Nagog Park|Elm Saint|P\.O\. Box|Peaks Cv|Ne Brazee',"",match1)
            match1 = re.sub('Se, |Highway North|Rd Saint S|El Camino Real|.+?Management|E Eubank|.+?Product ',"",match1)
            match1 = re.sub('El Camino Real|.+?(Tax |Tax,)|Circle,|Northwestern Hgy|Rr |Putnam Saint|CO.+?Hill',"",match1)
            match1 = re.sub('Bissonnet Saint|S[ew] |West Tenth Stree|S E|Camino Del Tomasini|The Alameda',"",match1)
            match1 = re.sub('North Central Expressway|W Tenth Saint|Saint ste|W. Cummings Park|Prosper Saint',"",match1)
            match1 = re.sub('Glenlake Ave|Namco|N\. Central Ave|suite |N. Military Trail|.+?Alton Pl|Ludelle Saint',"",match1)
            match1 = re.sub('S\.W\. Terr|Dtn, Llc|.+?East Marcy|W Micheltorena|N Thompson Saint|CO Jos M Glickstein',"",match1)
            match1 = re.sub('S. Whittle Ave|S Orange Blossom Tr|Rt\.|Big Duke Trl|Von Karman \(\)|Hwy [A-Z] ',"",match1)
            match1 = re.sub('E Main Saint|Station, |Toledo Saint  A|Number |[EWNS] |ste |Woodlawn Av|Knox Av',"",match1)
            match1 = re.sub('depere, Wi',"De Pere, Wi",match1)
            match1 = re.sub('ScifAnnapolis Junction, Md,',"Annapolis Junction Md",match1)
                        
            ### capitalize the first letter in all strings, remove punctuation and whitespace around the
            ### the city-state strings. States were converted to two-letter abbreviations
            match1 = match1.title()
            match1 = match1.translate(str.maketrans('','',string.punctuation))
            match1 = match1.lstrip().rstrip()
            match1 = re.sub("California","Ca",match1)
            match1 = re.sub("Connecticut","Ct",match1)
            match1 = re.sub("Massachusetts|Massachusett","Ma",match1)
            match1 = re.sub("Nebraska","Ne",match1)
            match1 = re.sub("Florida","Fl",match1)
            match1 = re.sub("Georgia","Ga",match1)
            match1 = re.sub("Washington","Wa",match1)
            match1 = re.sub(" New York"," Ny",match1)
            match1 = re.sub("Delaware","De",match1)
            match1 = re.sub("Tennessee","Tn",match1)
            match1 = re.sub("Missouri","Mo",match1)
            match1 = re.sub("Texas","Tx",match1)
            match1 = re.sub("Indiana","In",match1)
            match1 = re.sub("Pennsylvania","Pa",match1)
            match1 = re.sub("Oregon","Or",match1)
            match1 = re.sub("Virginia","Va",match1)
            match1 = re.sub("Illinois|Illinoi","Il",match1)
            match1 = re.sub("Kentucky","Ky",match1)
            match1 = re.sub("North Carolina","Nc",match1)
            match1 = re.sub("New Jersey","Nj",match1)
            match1 = re.sub("Colorado","Co",match1)
            match1 = re.sub("Maryland","Md",match1)
            match1 = re.sub("Ohio","Oh",match1)
            match1 = re.sub("Arizona","Az",match1)
            match1 = re.sub("Nevada","Nv",match1)
            match1 = re.sub("Utah","Ut",match1)
            match1 = re.sub("Michigan","Mi",match1)
            match1 = re.sub("New Hampshire","Nh",match1)
            match1 = re.sub("Vermont","Vt",match1)
            match1 = re.sub("Kansas","Ks",match1)
            match1 = re.sub("Oklahoma","Ok",match1)
            match1 = re.sub("Iowa","Ia",match1)
            match1 = re.sub("Louisiana","La",match1)
            match1 = re.sub("Rhode Island","Ri",match1)
            match1 = re.sub("Wisconsin","Wi",match1)
            match1 = re.sub("Hawaii","Hi",match1)
            match1 = re.sub("Montana","Mt",match1)
            match1 = re.sub("District Of Columbia","Dc",match1)
            match1 = re.sub("West Virginia","Wv",match1)
            match1 = re.sub("Alabama","Al",match1)
            match1 = re.sub("Idaho","Id",match1)
            match1 = re.sub("Maine","Me",match1)
            match1 = re.sub("New Mexico","Nm",match1)
            match1 = re.sub("South Carolina","Sc",match1)
            match1 = re.sub("North Dakota","Nd",match1)
            match1 = re.sub("South Dakota","Sd",match1)
            match1 = re.sub("Arkansas","Ar",match1)
            match1 = re.sub("Alaska","Ak",match1)
            match1 = re.sub("Wyoming","Wy",match1)
            match1 = re.sub("Mississippi","Ms",match1)
            match1 = re.sub("Minnesota","Mn",match1)  
            match1 = re.sub("Virginia","Va",match1)  

            ### final set of cleaning for the city-state pairs prior to adding them to city list and
            ### state list that will be appended to the input dataframe
            match1 = re.sub('Inpoli',"Indianapolis",match1)
            match1 = re.sub('Dalla Tx',"Dallas Tx",match1)
            match1 = re.sub('Fla',"Fl",match1)
            match1 = re.sub('Kipling Memphi Tn|Memphi Tn',"Memphis Tn",match1)
            match1 = re.sub('White Pln Ny',"White Plain Ny",match1)
            match1 = re.sub('X  Hyrum Ut',"Hyrum Ut",match1)
            match1 = re.sub('Saint Loui Mo',"Saint Louis Mo",match1)
            match1 = re.sub('Jenner   Irvine Ca',"Irvine Ca",match1)
            match1 = re.sub('Los Angele Ca|Nh   Los Angeles Ca',"Los Angeles Ca",match1)
            match1 = re.sub('N West Palm Beach Fl',"West Palm Beach Fl",match1)
            match1 = re.sub('Y  Pasadena Tx',"Pasadena Tx",match1)
            match1 = re.sub('S Chicago Il',"Chicago Il",match1)
            match1 = re.sub('J Houston Tx',"Houston Tx",match1)
            match1 = re.sub('R Ny Ny|Saint Ny Ny',"New York Ny",match1)
            match1 = re.sub('Mission Wood K',"Mission Wood Ks",match1)
            match1 = re.sub('Saint Charles Mo|Saint Charle Mo',"Saint Charles Mo",match1)
            match1 = re.sub('Ne Oh',"Oh",match1)
            match1 = re.sub('Brooklyn Ny Ny',"Brooklyn Ny",match1)
            match1 = re.sub('F Austin Tx',"Austin Tx",match1)
            match1 = re.sub('Co Spring Co',"Colorado Springs Co",match1)
            
            ### uses the stateList variable to extract the states from the record. The join function is
            ### applied to prevent brackets added to the data frame in later steps
            sub_states1.append(''.join(re.findall(stateList,match1)))

            ### there are records where only the city is present. This if statement evaluates the
            ### length of the sub_states1 variable. A length equal to 0 corresponds to a record where no
            ### state is associated, only a city name. The string is appended to the sub_city variable.
            ### A length greater than 0 corresponds to a record that has a city-state pair and the
            ### string is sliced to remove the two-letter state and space between the city-state pair.
            ### The result is appended to the sub_city variable
            if len(sub_states1) == 0:
                sub_city.append(match1)
            
            else:
                sub_city.append(match1[:-3])

    ### the city and state is appended to a new list that will be added to the input dataframe. The lists
    ### in the nested for loop above were emptied and recycled for the next round of records.
    states.append(sub_states1)
    city.append(sub_city)
    sub_states1 = []
    sub_city = []

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### add the city and state lists to the input data frame and select the required columns
OC_results4['data_city']=city
OC_results4['data_state']=states

OC_results5=OC_results4.iloc[:,[0,1,2,3,4,5,7,8,9,10,11,12]]

### print general stats and first 5 records for dataset
display(OC_results5.info(),OC_results5.head())

Total time is 0.014085 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 2 to 2622
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   assignee_id              2034 non-null   object
 1   name                     2034 non-null   object
 2   jurisdiction_code        2034 non-null   object
 3   incorporation_date       2034 non-null   object
 4   alternative_names_clean  2034 non-null   object
 5   previous_names_clean     2034 non-null   object
 6   address_city             976 non-null    object
 7   address_state            961 non-null    object
 8   agent_city               292 non-null    object
 9   agent_state              290 non-null    object
 10  data_city                2034 non-null   object
 11  data_state               2034 non-null   object
dtypes: object(12)
memory usage: 206.6+ KB


None

,assignee_id,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state
2,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ak,1/3/2000,[],[Lendingtree],CHARLOTTE,NC,NaN,NaN,[],[]
3,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ut,6/27/1997,"[Servicemagic Home Loans, RealestateCom Realto...","[Creditsource, Lendingtree]",CHARLOTTE,NC,NaN,NaN,[],[]
4,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_la,8/29/1997,[],"[Lendingtree, Creditsource]",BATON ROUGE,LA,NaN,NaN,"[Charlotte, Charlotte]","[Nc, Nc]"
5,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_me,6/7/1996,"[RealestateCom Cancelled, GetsmartCom, Service...",[],NaN,NaN,NaN,NaN,[Charlotte],[Nc]
6,52e38962-9bfc-4082-b0c8-b7ba4ac04d83,Lendingtree,us_ut,6/27/1997,"[Getsmart, Servicemagic Real Estate, GetsmartC...","[Lendingtree, Creditsource]",CHARLOTTE,NC,NaN,NaN,[],[]


# Perform Direct Merge Between the PatentsView and OpenCorporates Results and Calculate the Number of Unique Assignee IDs

In [16]:
### perform a merge with the full dataset curated from PatentsView and the prepared data from the OpenCorporates
### results using the assignee_ids and organization names
mergeFullWithNoStates=fullData.merge(OC_results5,left_on=['assignee_id','organization'],
                                     right_on=['assignee_id','name'],how='inner')

display(mergeFullWithNoStates.info(),mergeFullWithNoStates.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1234 entries, 0 to 1233
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       1234 non-null   int64 
 1   assignee_id              1234 non-null   object
 2   location_id              1234 non-null   object
 3   organization             1234 non-null   object
 4   city                     1224 non-null   object
 5   state                    1220 non-null   object
 6   dateOfFirstPat           1234 non-null   object
 7   name                     1234 non-null   object
 8   jurisdiction_code        1234 non-null   object
 9   incorporation_date       1234 non-null   object
 10  alternative_names_clean  1234 non-null   object
 11  previous_names_clean     1234 non-null   object
 12  address_city             555 non-null    object
 13  address_state            548 non-null    object
 14  agent_city               208 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,VA,1/30/2001,The Egg Factory,us_va,12/22/1997,[],[],ROANOKE,Virginia,ROANOKE,Virginia,[],[]
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,MA,10/16/2018,Tank Vision,us_ri,1/8/2019,[],[],VINEYARD HAVEN,MA,PROVIDENCE,RI,[],[]
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,MA,10/16/2018,Tank Vision,us_de,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[]
3,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,MA,6/27/2017,Infinibox,us_de,1/31/2013,[],[],NaN,NaN,NaN,NaN,[],[]
4,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,NV,3/15/1983,Agricultural Aviation Engineering Company,us_ca,5/1/1958,[],[],LAS VEGAS,NV,NaN,NaN,[Las Vegas],[Nv]


In [17]:
### convert the city names and states to capitalize the first character in the string
mergeFullWithNoStates['state']=mergeFullWithNoStates['state'].str.title()
mergeFullWithNoStates['address_city']=mergeFullWithNoStates['address_city'].str.title()
mergeFullWithNoStates['address_state']=mergeFullWithNoStates['address_state'].str.title()
mergeFullWithNoStates['agent_city']=mergeFullWithNoStates['agent_city'].str.title()
mergeFullWithNoStates['agent_state']=mergeFullWithNoStates['agent_state'].str.title()
mergeFullWithNoStates['agent_state']=mergeFullWithNoStates['agent_state'].str.title()

display(mergeFullWithNoStates.info(),mergeFullWithNoStates.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1234 entries, 0 to 1233
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       1234 non-null   int64 
 1   assignee_id              1234 non-null   object
 2   location_id              1234 non-null   object
 3   organization             1234 non-null   object
 4   city                     1224 non-null   object
 5   state                    1220 non-null   object
 6   dateOfFirstPat           1234 non-null   object
 7   name                     1234 non-null   object
 8   jurisdiction_code        1234 non-null   object
 9   incorporation_date       1234 non-null   object
 10  alternative_names_clean  1234 non-null   object
 11  previous_names_clean     1234 non-null   object
 12  address_city             555 non-null    object
 13  address_state            548 non-null    object
 14  agent_city               208 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,us_va,12/22/1997,[],[],Roanoke,Virginia,Roanoke,Virginia,[],[]
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,us_ri,1/8/2019,[],[],Vineyard Haven,Ma,Providence,Ri,[],[]
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,us_de,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[]
3,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,Infinibox,us_de,1/31/2013,[],[],NaN,NaN,NaN,NaN,[],[]
4,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,Agricultural Aviation Engineering Company,us_ca,5/1/1958,[],[],Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv]


In [18]:
### calculate the number of unique assignee_ids
mergeFullWithNoStates['assignee_id'].nunique()

285

In [19]:
### filter out records that do not have a us_ prefix under the jurisdiction_code column. Once filtered,
### the state was extracted from the feature and saved under a new feature called subJurisCode, followed
### by selecting the fields for further inspection
mergeFullWithNoStates2=mergeFullWithNoStates[mergeFullWithNoStates['jurisdiction_code'].str.contains('us_',na=False)]
mergeFullWithNoStates2['subJurisCode']=mergeFullWithNoStates2['jurisdiction_code'].str.slice(3).str.title()
mergeFullWithNoStates3=mergeFullWithNoStates2.iloc[:,[0,1,2,3,4,5,6,7,18,9,10,11,12,13,14,15,16,17]]

### print general stats and first 5 records for dataset
display(mergeFullWithNoStates3.info(),mergeFullWithNoStates3.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1233
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       1205 non-null   int64 
 1   assignee_id              1205 non-null   object
 2   location_id              1205 non-null   object
 3   organization             1205 non-null   object
 4   city                     1195 non-null   object
 5   state                    1194 non-null   object
 6   dateOfFirstPat           1205 non-null   object
 7   name                     1205 non-null   object
 8   subJurisCode             1205 non-null   object
 9   incorporation_date       1205 non-null   object
 10  alternative_names_clean  1205 non-null   object
 11  previous_names_clean     1205 non-null   object
 12  address_city             548 non-null    object
 13  address_state            541 non-null    object
 14  agent_city               208 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,subJurisCode,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,Va,12/22/1997,[],[],Roanoke,Virginia,Roanoke,Virginia,[],[]
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,Ri,1/8/2019,[],[],Vineyard Haven,Ma,Providence,Ri,[],[]
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,De,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[]
3,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,Infinibox,De,1/31/2013,[],[],NaN,NaN,NaN,NaN,[],[]
4,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,Agricultural Aviation Engineering Company,Ca,5/1/1958,[],[],Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv]


In [20]:
### calculate the number of unique assignee_ids
mergeFullWithNoStates3['assignee_id'].nunique()

279

In [21]:
### feature construction was done by comparing the state feature from PatentsView to the state under
### the subJurisCode feature. If the states match, the record was given a code of zero; if the states
### do not match AND the subJurisCode state was not Delaware, the record was given a code of one; and
### if any record had a subJurisCode equal to Delaware, the record was given a code of two
mergeFullWithNoStates3['stateMatch']=np.where(mergeFullWithNoStates3['state'] == mergeFullWithNoStates3['subJurisCode'],0,
                                       np.where((mergeFullWithNoStates3['state'] != mergeFullWithNoStates3['subJurisCode']) &
                                                (mergeFullWithNoStates3['subJurisCode'] != 'De'),1,2))

display(mergeFullWithNoStates3.info(),mergeFullWithNoStates3.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1205 entries, 0 to 1233
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       1205 non-null   int64 
 1   assignee_id              1205 non-null   object
 2   location_id              1205 non-null   object
 3   organization             1205 non-null   object
 4   city                     1195 non-null   object
 5   state                    1194 non-null   object
 6   dateOfFirstPat           1205 non-null   object
 7   name                     1205 non-null   object
 8   subJurisCode             1205 non-null   object
 9   incorporation_date       1205 non-null   object
 10  alternative_names_clean  1205 non-null   object
 11  previous_names_clean     1205 non-null   object
 12  address_city             548 non-null    object
 13  address_state            541 non-null    object
 14  agent_city               208 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,subJurisCode,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state,stateMatch
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,Va,12/22/1997,[],[],Roanoke,Virginia,Roanoke,Virginia,[],[],0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,Ri,1/8/2019,[],[],Vineyard Haven,Ma,Providence,Ri,[],[],1
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,De,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[],2
3,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,Infinibox,De,1/31/2013,[],[],NaN,NaN,NaN,NaN,[],[],2
4,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,Agricultural Aviation Engineering Company,Ca,5/1/1958,[],[],Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv],1


In [22]:
### remove any duplicates that exist in the data
mergeFullWithNoStates4=mergeFullWithNoStates3.drop_duplicates(subset=['ID','assignee_id','location_id','organization',
                                                                      'state','subJurisCode'],keep='first')

### print general stats and first 5 records for dataset
display(mergeFullWithNoStates4.info(),mergeFullWithNoStates4.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1145 entries, 0 to 1233
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       1145 non-null   int64 
 1   assignee_id              1145 non-null   object
 2   location_id              1145 non-null   object
 3   organization             1145 non-null   object
 4   city                     1135 non-null   object
 5   state                    1134 non-null   object
 6   dateOfFirstPat           1145 non-null   object
 7   name                     1145 non-null   object
 8   subJurisCode             1145 non-null   object
 9   incorporation_date       1145 non-null   object
 10  alternative_names_clean  1145 non-null   object
 11  previous_names_clean     1145 non-null   object
 12  address_city             512 non-null    object
 13  address_state            505 non-null    object
 14  agent_city               198 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,subJurisCode,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state,stateMatch
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,Va,12/22/1997,[],[],Roanoke,Virginia,Roanoke,Virginia,[],[],0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,Ri,1/8/2019,[],[],Vineyard Haven,Ma,Providence,Ri,[],[],1
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,De,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[],2
3,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,Infinibox,De,1/31/2013,[],[],NaN,NaN,NaN,NaN,[],[],2
4,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,Agricultural Aviation Engineering Company,Ca,5/1/1958,[],[],Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv],1


In [23]:
### calculate the number for each category under the stateMatch column (e.g., 0, 1, or 2)
print("# of records with matching states (0):     ", mergeFullWithNoStates4['stateMatch'].value_counts()[0])
print("# of records with non-matching states (1): ", mergeFullWithNoStates4['stateMatch'].value_counts()[1])
print("# of records with matching to Delaware (2):", mergeFullWithNoStates4['stateMatch'].value_counts()[2])
print("The total number of records:               ", mergeFullWithNoStates4['stateMatch'].count())

# of records with matching states (0):      83
# of records with non-matching states (1):  909
# of records with matching to Delaware (2): 153
The total number of records:                1145


In [24]:
### filtered the records that were non-matching states
mergeFullWithNoStates5=mergeFullWithNoStates4.loc[(mergeFullWithNoStates4['stateMatch'] == 0) | 
                                    (mergeFullWithNoStates4['stateMatch'] == 2)].reset_index(drop=True)

### print general stats and first 5 records for dataset
display(mergeFullWithNoStates5.info(),mergeFullWithNoStates5.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       236 non-null    int64 
 1   assignee_id              236 non-null    object
 2   location_id              236 non-null    object
 3   organization             236 non-null    object
 4   city                     235 non-null    object
 5   state                    235 non-null    object
 6   dateOfFirstPat           236 non-null    object
 7   name                     236 non-null    object
 8   subJurisCode             236 non-null    object
 9   incorporation_date       236 non-null    object
 10  alternative_names_clean  236 non-null    object
 11  previous_names_clean     236 non-null    object
 12  address_city             47 non-null     object
 13  address_state            46 non-null     object
 14  agent_city               15 non-null     o

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,subJurisCode,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state,stateMatch
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,Va,12/22/1997,[],[],Roanoke,Virginia,Roanoke,Virginia,[],[],0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,De,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[],2
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,Infinibox,De,1/31/2013,[],[],NaN,NaN,NaN,NaN,[],[],2
3,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,7/26/2016,Fanamana,De,1/2/2013,[],[],NaN,NaN,NaN,NaN,[],[],2
4,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,4/20/2004,Pendpac,Ok,8/7/2003,[],[],NaN,NaN,NaN,NaN,[],[],0


In [25]:
### calculate the number of unique assignee_ids
mergeFullWithNoStates5['assignee_id'].nunique()

168

# Perform a Fuzzy Match Between the PatentsView Organization Name and the Alternative_Names_Clean and Previous_Names_Clean Fields

In [27]:
### perform a merge between the OC results and the full dataset to attach the PatentsView city and state
### to prepare the data for fuzzy matching between the two datasets
assingeeIDMerge=fullData.merge(OC_results5,on=['assignee_id'],how='inner')
assingeeIDMerge['city'].replace("St\. ","Saint ",regex=True,inplace=True)
assingeeIDMerge['address_city'].replace("St\. |St ","Saint ",regex=True,inplace=True)

### print general stats and first 5 records for dataset
display(assingeeIDMerge.info(),assingeeIDMerge.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       2849 non-null   int64 
 1   assignee_id              2849 non-null   object
 2   location_id              2849 non-null   object
 3   organization             2849 non-null   object
 4   city                     2817 non-null   object
 5   state                    2818 non-null   object
 6   dateOfFirstPat           2849 non-null   object
 7   name                     2849 non-null   object
 8   jurisdiction_code        2849 non-null   object
 9   incorporation_date       2849 non-null   object
 10  alternative_names_clean  2849 non-null   object
 11  previous_names_clean     2849 non-null   object
 12  address_city             1346 non-null   object
 13  address_state            1318 non-null   object
 14  agent_city               399 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,VA,1/30/2001,The Egg Factory,us_va,12/22/1997,[],[],ROANOKE,Virginia,ROANOKE,Virginia,[],[]
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,MA,10/16/2018,The Vision Tank,us_de,4/19/2005,[],[],NaN,NaN,NaN,NaN,[],[]
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,MA,10/16/2018,Tank Vision Environmental,us_ca,6/23/1994,[],[],ANAHEIM,CA,NaN,NaN,[Irvine],[Ca]
3,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,MA,10/16/2018,Tank Vision,us_ri,1/8/2019,[],[],VINEYARD HAVEN,MA,PROVIDENCE,RI,[],[]
4,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,MA,10/16/2018,Tank Vision,us_de,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[]


In [28]:
### convert the city names and states to capitalize the first character in the string
assingeeIDMerge['state']=assingeeIDMerge['state'].str.title()
assingeeIDMerge['address_city']=assingeeIDMerge['address_city'].str.title()
assingeeIDMerge['address_state']=assingeeIDMerge['address_state'].str.lower()
assingeeIDMerge['agent_city']=assingeeIDMerge['agent_city'].str.title()
assingeeIDMerge['agent_state']=assingeeIDMerge['agent_state'].str.lower()

#convert states from full names to two-letter abbreviations under the address_state and agent_state features
assingeeIDMerge.replace({'address_state':{"california":"ca","connecticut":"ct","massachusetts":"ma","florida":"fl",
                                          "georgia":"ga","washington":"wa","new york":"ny","delaware":"de","tennessee":"tn",
                                          "missouri":"mo","texas":"tx","indiana":"in","minnesota":"mn","pennsylvania":"pa",
                                          "oregon":"or","virginia":"va","illinois":"il","kentucky":"ky","north carolina":"nc",
                                          "new jersey":"nj","colorado":"co","maryland":"md","ohio":"oh","arizona":"az",
                                          "nevada":"nv","utah":"ut","michigan":"mi","new hampshire":"nh","vermont":"vt",
                                          "kansas":"ks","oklahoma":"ok","iowa":"ia","louisiana":"la","rhode island":"ri",
                                          "wisconsin":"wi","hawaii":"hi","montana":"mt","nebraska":"ne",
                                          "district of columbia":"dc","west virginia":"wv","alabama":"al","idaho":"id",
                                          "maine":"me","new mexico":"nm","south carolina":"sc","north dakota":"nd",
                                          "south dakota":"sd","arkansas":"ar","alaska":"ak","wyoming":"wy",
                                          "mississippi":"ms"}},inplace=True)

assingeeIDMerge.replace({'agent_state':{"california":"ca","connecticut":"ct","massachusetts":"ma","florida":"fl",
                                          "georgia":"ga","washington":"wa","new york":"ny","delaware":"de","tennessee":"tn",
                                          "missouri":"mo","texas":"tx","indiana":"in","minnesota":"mn","pennsylvania":"pa",
                                          "oregon":"or","virginia":"va","illinois":"il","kentucky":"ky","north carolina":"nc",
                                          "new jersey":"nj","colorado":"co","maryland":"md","ohio":"oh","arizona":"az",
                                          "nevada":"nv","utah":"ut","michigan":"mi","new hampshire":"nh","vermont":"vt",
                                          "kansas":"ks","oklahoma":"ok","iowa":"ia","louisiana":"la","rhode island":"ri",
                                          "wisconsin":"wi","hawaii":"hi","montana":"mt","nebraska":"ne",
                                          "district of columbia":"dc","west virginia":"wv","alabama":"al","idaho":"id",
                                          "maine":"me","new mexico":"nm","south carolina":"sc","north dakota":"nd",
                                          "south dakota":"sd","arkansas":"ar","alaska":"ak","wyoming":"wy",
                                          "mississippi":"ms"}},inplace=True)

### convert the first letter in each string to Uppercase and lowercase the remaining strings
assingeeIDMerge['address_state']=assingeeIDMerge['address_state'].str.title()
assingeeIDMerge['agent_state']=assingeeIDMerge['agent_state'].str.title()

### print general stats and first 5 records for dataset
display(assingeeIDMerge.info(),assingeeIDMerge.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       2849 non-null   int64 
 1   assignee_id              2849 non-null   object
 2   location_id              2849 non-null   object
 3   organization             2849 non-null   object
 4   city                     2817 non-null   object
 5   state                    2818 non-null   object
 6   dateOfFirstPat           2849 non-null   object
 7   name                     2849 non-null   object
 8   jurisdiction_code        2849 non-null   object
 9   incorporation_date       2849 non-null   object
 10  alternative_names_clean  2849 non-null   object
 11  previous_names_clean     2849 non-null   object
 12  address_city             1346 non-null   object
 13  address_state            1318 non-null   object
 14  agent_city               399 non-null   

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,us_va,12/22/1997,[],[],Roanoke,Va,Roanoke,Va,[],[]
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,The Vision Tank,us_de,4/19/2005,[],[],NaN,NaN,NaN,NaN,[],[]
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision Environmental,us_ca,6/23/1994,[],[],Anaheim,Ca,NaN,NaN,[Irvine],[Ca]
3,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,us_ri,1/8/2019,[],[],Vineyard Haven,Ma,Providence,Ri,[],[]
4,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,us_de,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[]


In [29]:
### import the libraries required for fuzzy matching; scoring is between the organization names in PatentsView
### against the name, alternative_names_clean, and previous_names_clean features in the OpenCoporates results
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

### start timer
t0=time.time()

### determine the length of the data frame and create 2 empty lists
aa=len(assingeeIDMerge)
mat1=[]
mat2=[]

for i in range(aa):

    ### try/except is used to bypass cells with an NaN. Removing this will force the user
    ### to deal with exception errors
    try:
        ### calculate the fuzzy score between the organization name in PatentsView and name
        ### from the OpenCorporates results for further evaluation
        q=fuzz.ratio(assingeeIDMerge.iloc[i,3], assingeeIDMerge.iloc[i,7])

        ### most simplest and most representative in the data; if the score is 100, it is
        ### a perfect match, otherwise, the remaining code will resolve score less than 100
        if q == 100:
            mat1.append(q)
            mat2.append(assingeeIDMerge.iloc[i,3])
        
        ### this section resolves all q values not equal to 100
        elif ( q != 100 ):

            ### calculate the length of the data in each row for the alternative_names_clean
            ### column and previous_names_clean column
            if len(assingeeIDMerge.iloc[i,10]) == 0:
                r=0
            
            elif len(assingeeIDMerge.iloc[i,10]) > 0:
                r=len(assingeeIDMerge.iloc[i,10])
            
            if len(assingeeIDMerge.iloc[i,11]) == 0:
                d=0
            
            elif len(assingeeIDMerge.iloc[i,11]) > 0:
                d=len(assingeeIDMerge.iloc[i,11])
                        
            ### the remaining parts of the code uses if statements to step through the many
            ### conditions that may be present in the data. As each condition is satisfied,
            ### the mat1 and mat2 lists are appended with the data; take note that q is the
            ### original score and s, v, e, and f are separate scores that are compared
            ### against q. The first if statement evaluates the lengths of r and q, and if
            ### both are zero, skips the remaining code and appends mat1 and mat2 with the
            ### fuzzy score and organization name from OpenCorporates as the top hit
            if r == 0 and d == 0:
                mat1.append(q)
                mat2.append(assingeeIDMerge['name'][i])
                                         
            ### calculate the score for the alternative_names_clean and previous_names_clean
            ### columns. Once calculated, they are compared against each other and q to
            ### determine the score that is highest. The 'best' score is appened to mat1 and
            ### the name of the organization is appended to mat2. The same scorer is utilized 
            ### as the above but the process.extractOne function retrieves the organization
            ### match with the highest score and saves it as a tuple
            elif r == 1 and d == 1:
                s=process.extractOne(assingeeIDMerge['organization'][i],
                                     assingeeIDMerge['alternative_names_clean'][i],scorer=fuzz.ratio)
                e=process.extractOne(assingeeIDMerge['organization'][i],
                                     assingeeIDMerge['previous_names_clean'][i],scorer=fuzz.ratio)
                
                if s[1] > e[1] and s[1] > q:
                    mat1.append(s[1])
                    mat2.append(s[0])
                
                elif e[1] > s[1] and e[1] > q:
                    mat1.append(e[1])
                    mat2.append(e[0])
                
                elif e[1] == q or s[1] == q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
                
                else:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
            
            ### this section is the same as above, except the code is looking at the
            ### alternative_names_clean column only
            elif r == 1:
                s=process.extractOne(assingeeIDMerge['organization'][i],
                                     assingeeIDMerge['alternative_names_clean'][i],scorer=fuzz.ratio)

                if s[1] > q:
                    mat1.append(s[1])
                    mat2.append(s[0])
                
                elif s[1] == q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
                    
                elif s[1] < q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
            
            ### this section is the same as above, except the code is looking at the
            ### previous_names_clean column only
            elif d == 1:
                e=process.extractOne(assingeeIDMerge['organization'][i],
                                     assingeeIDMerge['previous_names_clean'][i],scorer=fuzz.ratio)

                if e[1] > q:
                    mat1.append(e[1])
                    mat2.append(e[0])
                
                elif e[1] == q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
                    
                elif e[1] < q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])      
            
            ## the following two sections resolve r and d lengths greater than 1 (i.e.,
            ## records that have more than 1 company names in the alternative_names_clean 
            ## and previous_names_clean columns)
            elif r > 1:
                v=process.extractOne(assingeeIDMerge['organization'][i],
                                     assingeeIDMerge['alternative_names_clean'][i],scorer=fuzz.ratio)                

                if v[1] > q:
                    mat1.append(v[1])
                    mat2.append(v[0])
                    
                elif v[1] == q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
                    
                elif v[1] < q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])
            
            elif d > 1:
                f=process.extractOne(assingeeIDMerge['organization'][i],
                                     assingeeIDMerge['previous_names_clean'][i],scorer=fuzz.ratio)

                if f[1] > q:
                    mat1.append(f[1])
                    mat2.append(f[0])
                    
                elif f[1] == q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])                    
                    
                elif f[1] < q:
                    mat1.append(q)
                    mat2.append(assingeeIDMerge['name'][i])

    except:
        ### error handling that places an NaN for every cells that does not have a value
        ### in the mat1 and/or mat2 lists
        mat1.append(np.nan)
        mat2.append(np.nan)

### print general stats and first 5 records for dataset
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

Total time is 0.005668 mins


In [30]:
### start timer
t0=time.time()

### combines mat1 and mat2 into a single dataframe
finalList=[list(w) for w in zip(mat1, mat2)]
finalListDf=pd.DataFrame(finalList,columns=['scores','names'])

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(finalListDf.info(),finalListDf.head(n=25))

Total time is 0.000033 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2849 entries, 0 to 2848
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   scores  2849 non-null   int64 
 1   names   2849 non-null   object
dtypes: int64(1), object(1)
memory usage: 44.6+ KB


None

,scores,names
0,100,The Egg Factory
1,62,The Vision Tank
2,61,Tank Vision Environmental
3,100,Tank Vision
4,100,Tank Vision
5,100,Infinibox
6,100,Agricultural Aviation Engineering Company
7,100,Fanamana
8,100,Fanamana
9,100,US Wind Farming


In [31]:
### start timer
t0=time.time()

### appends the original dataframe with the results from the scoring steps in the preceeding code blocks
assingeeIDMerge['nameScores'] = finalListDf['scores']
assingeeIDMerge['matchNames'] = finalListDf['names']

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assingeeIDMerge.info(),assingeeIDMerge.head())

Total time is 0.000034 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       2849 non-null   int64 
 1   assignee_id              2849 non-null   object
 2   location_id              2849 non-null   object
 3   organization             2849 non-null   object
 4   city                     2817 non-null   object
 5   state                    2818 non-null   object
 6   dateOfFirstPat           2849 non-null   object
 7   name                     2849 non-null   object
 8   jurisdiction_code        2849 non-null   object
 9   incorporation_date       2849 non-null   object
 10  alternative_names_clean  2849 non-null   object
 11  previous_names_clean     2849 non-null   object
 12  address_city             1346 non-null   object
 13  address_state            1318 non-null   object
 14  agent_city  

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,name,jurisdiction_code,incorporation_date,alternative_names_clean,previous_names_clean,address_city,address_state,agent_city,agent_state,data_city,data_state,nameScores,matchNames
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,The Egg Factory,us_va,12/22/1997,[],[],Roanoke,Va,Roanoke,Va,[],[],100,The Egg Factory
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,The Vision Tank,us_de,4/19/2005,[],[],NaN,NaN,NaN,NaN,[],[],62,The Vision Tank
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision Environmental,us_ca,6/23/1994,[],[],Anaheim,Ca,NaN,NaN,[Irvine],[Ca],61,Tank Vision Environmental
3,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,us_ri,1/8/2019,[],[],Vineyard Haven,Ma,Providence,Ri,[],[],100,Tank Vision
4,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,Tank Vision,us_de,1/3/2017,[],[],NaN,NaN,NaN,NaN,[],[],100,Tank Vision


In [32]:
### rearrange and select the features for further evaluation
assingeeIDMerge1=assingeeIDMerge
assingeeIDMerge1.drop(labels=['name','alternative_names_clean','previous_names_clean'],axis=1,inplace=True)

eigh_col = assingeeIDMerge1.pop('nameScores')
nine_col = assingeeIDMerge1.pop('matchNames')

assingeeIDMerge1.insert(8, 'nameScores', eigh_col)
assingeeIDMerge1.insert(9, 'matchNames', nine_col)

### print general stats and first 5 records for dataset
display(assingeeIDMerge1.info(),assingeeIDMerge1.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2849 entries, 0 to 2848
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2849 non-null   int64 
 1   assignee_id         2849 non-null   object
 2   location_id         2849 non-null   object
 3   organization        2849 non-null   object
 4   city                2817 non-null   object
 5   state               2818 non-null   object
 6   dateOfFirstPat      2849 non-null   object
 7   jurisdiction_code   2849 non-null   object
 8   nameScores          2849 non-null   int64 
 9   matchNames          2849 non-null   object
 10  incorporation_date  2849 non-null   object
 11  address_city        1346 non-null   object
 12  address_state       1318 non-null   object
 13  agent_city          399 non-null    object
 14  agent_state         397 non-null    object
 15  data_city           2849 non-null   object
 16  data_state          2849

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,jurisdiction_code,nameScores,matchNames,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,us_va,100,The Egg Factory,12/22/1997,Roanoke,Va,Roanoke,Va,[],[]
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,us_de,62,The Vision Tank,4/19/2005,NaN,NaN,NaN,NaN,[],[]
2,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,us_ca,61,Tank Vision Environmental,6/23/1994,Anaheim,Ca,NaN,NaN,[Irvine],[Ca]
3,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,us_ri,100,Tank Vision,1/8/2019,Vineyard Haven,Ma,Providence,Ri,[],[]
4,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,us_de,100,Tank Vision,1/3/2017,NaN,NaN,NaN,NaN,[],[]


# Identify the Number of Records Between the Datasets with the Same State and Those with Delaware

In [34]:
### group the data by assignee_id and select the record with the highest fuzzy score
assigneeScores=pd.DataFrame(assingeeIDMerge1.groupby(by=['assignee_id'],as_index=False)['nameScores'].max())

### print general stats and first 5 records for dataset
display(assigneeScores.info(),assigneeScores.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 410
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   assignee_id  411 non-null    object
 1   nameScores   411 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.6+ KB


None

,assignee_id,nameScores
0,00059b74-f414-43cb-a36e-e30fb41c9285,100
1,004053ef-9a48-4d23-9fd9-93ff47b7c299,100
2,00de4104-b930-4719-a790-18e59618ac05,100
3,0116426f-e05d-4c31-bed1-5e811c29400a,100
4,01756cf3-dcd4-4332-a3ff-a240e090c08f,100


In [35]:
### start timer
t0=time.time()

### perform an inner merge with the previous data frame constructed by the groupby function with the assingeeIDMerge1
### data using the assignee_id and nameScores as matching features, select the columns to keep, and sort the data
### by the ID
assigneeScores1=assigneeScores.merge(assingeeIDMerge1,on=['assignee_id','nameScores'],
                                     how='inner').iloc[:,[2,0,3,4,5,6,7,1,9,8,10,11,12,13,14,15,16]].sort_values(by=['ID'])

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assigneeScores1.info(),assigneeScores1.head())

Total time is 0.000083 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1636 entries, 1568 to 543
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1636 non-null   int64 
 1   assignee_id         1636 non-null   object
 2   location_id         1636 non-null   object
 3   organization        1636 non-null   object
 4   city                1608 non-null   object
 5   state               1607 non-null   object
 6   dateOfFirstPat      1636 non-null   object
 7   nameScores          1636 non-null   int64 
 8   matchNames          1636 non-null   object
 9   jurisdiction_code   1636 non-null   object
 10  incorporation_date  1636 non-null   object
 11  address_city        794 non-null    object
 12  address_state       787 non-null    object
 13  agent_city          278 non-null    object
 14  agent_state         276 non-null    object
 15  data_city           1636 non-null   object

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,jurisdiction_code,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state
1568,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,100,The Egg Factory,us_va,12/22/1997,Roanoke,Va,Roanoke,Va,[],[]
1564,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,us_de,1/3/2017,NaN,NaN,NaN,NaN,[],[]
1563,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,us_ri,1/8/2019,Vineyard Haven,Ma,Providence,Ri,[],[]
1536,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,100,Infinibox,us_de,1/31/2013,NaN,NaN,NaN,NaN,[],[]
1519,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,100,Agricultural Aviation Engineering Company,us_ca,5/1/1958,Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv]


In [36]:
### determine the number of unique assignee_ids in the data
assigneeScores1['assignee_id'].nunique()

411

In [37]:
### start timer
t0=time.time()

### remove non-US based organizations and construct a new feature containing the state for the record
### from the jurisdiction_code column
assigneeScores2=assigneeScores1[assigneeScores1['jurisdiction_code'].str.contains('us_',na=False)]
assigneeScores2['subJurisCode']=assigneeScores2['jurisdiction_code'].str.slice(3).str.title()
assigneeScores3=assigneeScores2.iloc[:,[0,1,2,3,4,5,6,7,8,17,10,11,12,13,14,15,16]]

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assigneeScores3.info(),assigneeScores3.head())

Total time is 0.000083 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1558 entries, 1568 to 543
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1558 non-null   int64 
 1   assignee_id         1558 non-null   object
 2   location_id         1558 non-null   object
 3   organization        1558 non-null   object
 4   city                1532 non-null   object
 5   state               1534 non-null   object
 6   dateOfFirstPat      1558 non-null   object
 7   nameScores          1558 non-null   int64 
 8   matchNames          1558 non-null   object
 9   subJurisCode        1558 non-null   object
 10  incorporation_date  1558 non-null   object
 11  address_city        780 non-null    object
 12  address_state       773 non-null    object
 13  agent_city          278 non-null    object
 14  agent_state         276 non-null    object
 15  data_city           1558 non-null   object

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,subJurisCode,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state
1568,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,100,The Egg Factory,Va,12/22/1997,Roanoke,Va,Roanoke,Va,[],[]
1564,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,De,1/3/2017,NaN,NaN,NaN,NaN,[],[]
1563,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,Ri,1/8/2019,Vineyard Haven,Ma,Providence,Ri,[],[]
1536,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,100,Infinibox,De,1/31/2013,NaN,NaN,NaN,NaN,[],[]
1519,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,100,Agricultural Aviation Engineering Company,Ca,5/1/1958,Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv]


In [39]:
### start timer
t0=time.time()

### manual inspection indicates records with less than a 90% fuzzy score result in poor matches
### between the PatentsView and OpenCorporates data and were filtered from the data set
assigneeScores4=assigneeScores3.loc[assigneeScores3['nameScores'] >= 90]

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assigneeScores4.info(),assigneeScores4.head())

Total time is 0.000033 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1469 entries, 1568 to 543
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1469 non-null   int64 
 1   assignee_id         1469 non-null   object
 2   location_id         1469 non-null   object
 3   organization        1469 non-null   object
 4   city                1445 non-null   object
 5   state               1446 non-null   object
 6   dateOfFirstPat      1469 non-null   object
 7   nameScores          1469 non-null   int64 
 8   matchNames          1469 non-null   object
 9   subJurisCode        1469 non-null   object
 10  incorporation_date  1469 non-null   object
 11  address_city        725 non-null    object
 12  address_state       718 non-null    object
 13  agent_city          263 non-null    object
 14  agent_state         261 non-null    object
 15  data_city           1469 non-null   object

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,subJurisCode,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state
1568,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,100,The Egg Factory,Va,12/22/1997,Roanoke,Va,Roanoke,Va,[],[]
1564,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,De,1/3/2017,NaN,NaN,NaN,NaN,[],[]
1563,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,Ri,1/8/2019,Vineyard Haven,Ma,Providence,Ri,[],[]
1536,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,100,Infinibox,De,1/31/2013,NaN,NaN,NaN,NaN,[],[]
1519,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,100,Agricultural Aviation Engineering Company,Ca,5/1/1958,Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv]


In [40]:
### determine the number of unique assignee_ids in the data
assigneeScores4['assignee_id'].nunique()

329

In [41]:
### start timer
t0=time.time()

### feature construction was done by comparing the state feature from PatentsView to the state under
### the subJurisCode feature. If the states match, the record was given a code of zero; if the states
### do not match AND the subJurisCode state was not Delaware, the record was given a code of one; and
### if any record had a subJurisCode equal to Delaware, the record was given a code of two
assigneeScores4['stateMatch']=np.where(assigneeScores4['state'] == assigneeScores4['subJurisCode'],0,
                                       np.where((assigneeScores4['state'] != assigneeScores4['subJurisCode']) &
                                                (assigneeScores4['subJurisCode'] != 'De'),1,2))

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assigneeScores4.info(),assigneeScores4.head())

Total time is 0.000017 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1469 entries, 1568 to 543
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1469 non-null   int64 
 1   assignee_id         1469 non-null   object
 2   location_id         1469 non-null   object
 3   organization        1469 non-null   object
 4   city                1445 non-null   object
 5   state               1446 non-null   object
 6   dateOfFirstPat      1469 non-null   object
 7   nameScores          1469 non-null   int64 
 8   matchNames          1469 non-null   object
 9   subJurisCode        1469 non-null   object
 10  incorporation_date  1469 non-null   object
 11  address_city        725 non-null    object
 12  address_state       718 non-null    object
 13  agent_city          263 non-null    object
 14  agent_state         261 non-null    object
 15  data_city           1469 non-null   object

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,subJurisCode,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state,stateMatch
1568,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,100,The Egg Factory,Va,12/22/1997,Roanoke,Va,Roanoke,Va,[],[],0
1564,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,De,1/3/2017,NaN,NaN,NaN,NaN,[],[],2
1563,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,Ri,1/8/2019,Vineyard Haven,Ma,Providence,Ri,[],[],1
1536,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,100,Infinibox,De,1/31/2013,NaN,NaN,NaN,NaN,[],[],2
1519,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,100,Agricultural Aviation Engineering Company,Ca,5/1/1958,Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv],1


In [42]:
### start timer
t0=time.time()

### duplicates were dropped utilizing the features below
assigneeScores5=assigneeScores4.drop_duplicates(subset=['ID','assignee_id','location_id','organization',
                                                        'state','nameScores','subJurisCode'],keep='first')

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assigneeScores5.info(),assigneeScores5.head())

Total time is 0.000067 mins
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1402 entries, 1568 to 551
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  1402 non-null   int64 
 1   assignee_id         1402 non-null   object
 2   location_id         1402 non-null   object
 3   organization        1402 non-null   object
 4   city                1378 non-null   object
 5   state               1379 non-null   object
 6   dateOfFirstPat      1402 non-null   object
 7   nameScores          1402 non-null   int64 
 8   matchNames          1402 non-null   object
 9   subJurisCode        1402 non-null   object
 10  incorporation_date  1402 non-null   object
 11  address_city        684 non-null    object
 12  address_state       677 non-null    object
 13  agent_city          253 non-null    object
 14  agent_state         251 non-null    object
 15  data_city           1402 non-null   object

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,subJurisCode,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state,stateMatch
1568,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,100,The Egg Factory,Va,12/22/1997,Roanoke,Va,Roanoke,Va,[],[],0
1564,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,De,1/3/2017,NaN,NaN,NaN,NaN,[],[],2
1563,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,Ri,1/8/2019,Vineyard Haven,Ma,Providence,Ri,[],[],1
1536,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,100,Infinibox,De,1/31/2013,NaN,NaN,NaN,NaN,[],[],2
1519,2655,e62b4591-f071-4597-b791-35e9075d2af1,f97aecf0-cb90-11eb-9615-121df0c29c1e,Agricultural Aviation Engineering Company,Las Vegas,Nv,3/15/1983,100,Agricultural Aviation Engineering Company,Ca,5/1/1958,Las Vegas,Nv,NaN,NaN,[Las Vegas],[Nv],1


In [43]:
### calculate the number for each category under the stateMatch column (e.g., 0, 1, or 2)
print("# of records with matching states:     ", assigneeScores5['stateMatch'].value_counts()[0])
print("# of records with non-matching states: ", assigneeScores5['stateMatch'].value_counts()[1])
print("# of records with matching to Delaware:", assigneeScores5['stateMatch'].value_counts()[2])
print("The total number of records:           ", assigneeScores5['stateMatch'].count())

# of records with matching states:      134
# of records with non-matching states:  1109
# of records with matching to Delaware: 159
The total number of records:            1402


In [44]:
### start timer
t0=time.time()

### filter for records with a score of 0 or 2 in the stateMatch feature
assigneeScores6=assigneeScores5.loc[(assigneeScores5['stateMatch'] == 0) | 
                                    (assigneeScores5['stateMatch'] == 2)].reset_index(drop=True)

### end timer and print total time
t1=time.time()
total=t1-t0
print("Total time is %4f" % (total/60), "mins")

### print general stats and first 5 records for dataset
display(assigneeScores6.info(),assigneeScores6.head())

Total time is 0.000033 mins
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  293 non-null    int64 
 1   assignee_id         293 non-null    object
 2   location_id         293 non-null    object
 3   organization        293 non-null    object
 4   city                292 non-null    object
 5   state               292 non-null    object
 6   dateOfFirstPat      293 non-null    object
 7   nameScores          293 non-null    int64 
 8   matchNames          293 non-null    object
 9   subJurisCode        293 non-null    object
 10  incorporation_date  293 non-null    object
 11  address_city        88 non-null     object
 12  address_state       87 non-null     object
 13  agent_city          24 non-null     object
 14  agent_state         23 non-null     object
 15  data_city           293 non-null    object
 16

None

,ID,assignee_id,location_id,organization,city,state,dateOfFirstPat,nameScores,matchNames,subJurisCode,incorporation_date,address_city,address_state,agent_city,agent_state,data_city,data_state,stateMatch
0,875,f7b31db8-0a3d-425e-a79f-d84ba6333b60,de5d6510-cb90-11eb-9615-121df0c29c1e,The Egg Factory,Roanoke,Va,1/30/2001,100,The Egg Factory,Va,12/22/1997,Roanoke,Va,Roanoke,Va,[],[],0
1,1284,f3e6ae72-e2b1-4165-a0f7-9be773bf1e56,9447f283-cb8e-11eb-9615-121df0c29c1e,Tank Vision,Vineyard Haven,Ma,10/16/2018,100,Tank Vision,De,1/3/2017,NaN,NaN,NaN,NaN,[],[],2
2,1667,f0447e02-fb80-46f7-a97c-cb4b2f398aeb,4b56fc31-cb8e-11eb-9615-121df0c29c1e,Infinibox,Westwood,Ma,6/27/2017,100,Infinibox,De,1/31/2013,NaN,NaN,NaN,NaN,[],[],2
3,2729,e5614631-da49-4351-a6c6-8f81358b767d,fb7257e4-cb8f-11eb-9615-121df0c29c1e,Fanamana,Washington,Dc,7/26/2016,100,Fanamana,De,1/2/2013,NaN,NaN,NaN,NaN,[],[],2
4,3756,dbd08974-b2f8-4753-b720-27b2fdff8b65,419babc4-cb8e-11eb-9615-121df0c29c1e,Pendpac,Fairview,Ok,4/20/2004,100,Pendpac,Ok,8/7/2003,NaN,NaN,NaN,NaN,[],[],0


In [45]:
### calculate the number of unique assignee_ids
assigneeScores6['assignee_id'].nunique()

212